In [1]:
# !unzip 'dataset.zip'

In [7]:
!rm -rf '__MACOSX'

In [8]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F
import torch.optim as optim

In [49]:
df = pd.read_csv('labels.csv')
df

,id,label,path
0,bedt_0.jpg,bed,images/bedt_0.jpg
1,bedt_1.jpg,bed,images/bedt_1.jpg
2,bedt_2.jpg,bed,images/bedt_2.jpg
3,bedt_3.jpg,bed,images/bedt_3.jpg
4,bedt_4.jpg,bed,images/bedt_4.jpg
...,...,...,...
49086,fad_27070.jpg,Casual Shoes,images/fad_27070.jpg
49087,fad_16559.jpg,Flip Flops,images/fad_16559.jpg
49088,fad_29176.jpg,Tshirts,images/fad_29176.jpg
49089,fad_22512.jpg,Perfume and Body Mist,images/fad_22512.jpg


In [50]:
labels_to_drop = ['Kurtas','Heels', 'Sunglasses','Flip Flops','Briefs','Belts','Socks', 'Formal Shoes',
       'Perfume and Body Mist','Flats','Bra','Deodorant',
       'Nail Polish', 'Lipstick', 'Track Pants', 'Clutches',
       'Sweatshirts', 'Caps', 'Sweaters', 'Ties', 'Jackets',
       'Innerwear Vests', 'Kurtis', 'Tunics', 'Nightdress',
       'Leggings', 'Pendant', 'Capris', 'Necklace and Chains',
       'Lip Gloss', 'Night suits', 'Trunk', 'Skirts', 'Scarves', 'Ring',
       'Dupatta', 'Accessory Gift Set', 'Cufflinks', 'Kajal and Eyeliner',
       'Kurta Sets', 'Free Gifts', 'Stoles', 'Duffel Bag', 'Bangle',
       'Laptop Bag', 'Foundation and Primer', 'Sports Sandals',
       'Bracelet', 'Face Moisturisers', 'Lounge Pants', 'Jewellery Set',
       'Fragrance Gift Set', 'Highlighter and Blush', 'Boxers', 'Compact',
       'Lip Liner', 'Mobile Pouch', 'Messenger Bag', 'Eyeshadow',
       'Suspenders', 'Camisoles', 'Mufflers', 'Patiala', 'Lounge Shorts',
       'Jeggings', 'Stockings', 'Salwar', 'Churidar', 'Tracksuits',
       'Face Wash and Cleanser', 'Sunscreen', 'Shoe Accessories',
       'Bath Robe', 'Gloves', 'Hair Colour', 'Rain Jacket', 'Swimwear',
       'Waist Pouch', 'Travel Accessory', 'Baby Dolls', 'Jumpsuit',
       'Lip Care', 'Waistcoat', 'Basketballs', 'Mascara', 'Mask and Peel',
       'Rompers', 'Booties', 'Water Bottle', 'Rucksacks', 'Concealer',
       'Shapewear', 'Tights', 'Footballs', 'Blazers', 'Clothing Set',
       'Headband', 'Wristbands', 'Salwar and Dupatta', 'Body Lotion',
       'Eye Cream', 'Nail Essentials', 'Umbrellas', 'Shrug',
       'Face Scrub and Exfoliator', 'Toner', 'Nehru Jackets',
       'Lehenga Choli', 'Robe', 'Lip Plumper', 'Makeup Remover',
       'Beauty Accessory', 'Lounge Tshirts', 'Trolley Bag',
       'Tablet Sleeve', 'Hat', 'Face Serum and Gel', 'Key chain',
       'Ties and Cufflinks', 'Rain Trousers', 'Hair Accessory',
       'Mens Grooming Kit', 'Shoe Laces', 'Cushion Covers',
       'Body Wash and Scrub', 'Suits', 'Ipad', 'Earrings','Shorts',
        'Shirts','Watches', 'Sports Shoes','Handbags','sw', 
        'Wallets','Sandals','Trousers','Sarees', 'Tops', 
        'sofa','bed','Backpacks', 'Dresses', 'phone','table']

In [51]:
indices = []
for i, label in enumerate(df['label']):
  if label in labels_to_drop:
    indices.append(i)

In [52]:
df.drop(df.index[indices], inplace = True)

In [53]:
df['label'].value_counts().keys().values

array(['Tshirts', 'Casual Shoes', 'chair', 'Jeans'], dtype=object)

In [54]:
indices = []
for i, label in enumerate(df['label']):
  if label == 'Casual Shoes':
    indices.append(i)

In [55]:
indices_to_drop = np.random.choice(indices, 1846)

In [57]:
indices = []
for i, label in enumerate(df['label']):
  if label == 'Tshirts':
    indices.append(i)

In [58]:
indices_to_drop = np.random.choice(indices, 6070)

In [56]:
df.drop(df.index[indices_to_drop], inplace = True)

In [59]:
df.drop(df.index[indices_to_drop], inplace = True)

In [72]:
df.reset_index(drop = True, inplace = True)

In [73]:
df

,id,label,path
0,chairt_0.jpg,chair,images/chairt_0.jpg
1,chairt_1.jpg,chair,images/chairt_1.jpg
2,chairt_2.jpg,chair,images/chairt_2.jpg
3,chairt_3.jpg,chair,images/chairt_3.jpg
4,chairt_4.jpg,chair,images/chairt_4.jpg
...,...,...,...
6073,fad_29990.jpg,Tshirts,images/fad_29990.jpg
6074,fad_21819.jpg,Tshirts,images/fad_21819.jpg
6075,fad_24435.jpg,Tshirts,images/fad_24435.jpg
6076,fad_19584.jpg,Tshirts,images/fad_19584.jpg


In [74]:
df['label'].value_counts()

Tshirts         2981
Casual Shoes    1488
chair           1000
Jeans            609
Name: label, dtype: int64

In [75]:
df.to_csv('new_df.csv')

In [70]:
for filename in os.listdir('images'):
    if filename not in df['path']:
        f = os.path.join(filename)
        print(f)
        os.remove(f)

In [71]:
print (len([name for name in os.listdir('images')])  ) 

0


In [10]:
keys = list(df['label'].value_counts().keys().values)

In [11]:
label_dict = {}
for i, k in enumerate(keys):
  label_dict[k] = i

In [12]:
df['label'] = [int(label_dict[label]) for label in df['label']]

In [13]:
df = df.drop(df.index[[11342, 20852, 36969, 41044, 44667]])

In [14]:
df = df.reset_index(drop = True)
df['path'][11342]

'images/fad_21380.jpg'

In [15]:
df

,id,label,path
0,bedt_0.jpg,13,images/bedt_0.jpg
1,bedt_1.jpg,13,images/bedt_1.jpg
2,bedt_2.jpg,13,images/bedt_2.jpg
3,bedt_3.jpg,13,images/bedt_3.jpg
4,bedt_4.jpg,13,images/bedt_4.jpg
...,...,...,...
49081,fad_27070.jpg,2,images/fad_27070.jpg
49082,fad_16559.jpg,15,images/fad_16559.jpg
49083,fad_29176.jpg,0,images/fad_29176.jpg
49084,fad_22512.jpg,22,images/fad_22512.jpg


In [16]:
train_df = df[:48000]
test_df = df[48000:]

In [22]:
class sustain_dataset(Dataset):
    def __init__(self, df, transform=None):
        self.labels_df = df
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_file = self.labels_df['path'][idx]
        try:
          img = Image.open(img_file)
          img = img.convert('RGB')
          label = self.labels_df['label'][idx]
          img = transforms.ToTensor()(img)
          img = transforms.Resize((100, 100))(img)
          return img, label
        except:
          print(img_file)
          return 0, 0

In [23]:
train_dataset= sustain_dataset(df = train_df)
test_dataset= sustain_dataset(df = test_df)

In [24]:
img, label = train_dataset[0]

In [25]:
img.shape

torch.Size([3, 100, 100])

In [39]:
train_dataloader = DataLoader(train_dataset, batch_size=64,num_workers=8 ,shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=8 ,shuffle=True)

In [40]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else 'cpu')
if use_cuda:
    print("Using CUDA!")
else:
    print('Not using CUDA!!!')

Not using CUDA!!!


In [28]:
!wget 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth'

--2021-10-30 14:24:19--  https://download.pytorch.org/models/vgg16_bn-6c64b313.pth
Resolving download.pytorch.org (download.pytorch.org)... 2600:9000:21f6:2c00:d:607e:4540:93a1, 2600:9000:21f6:4800:d:607e:4540:93a1, 2600:9000:21f6:8e00:d:607e:4540:93a1, ...
Connecting to download.pytorch.org (download.pytorch.org)|2600:9000:21f6:2c00:d:607e:4540:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553507836 (528M) [application/octet-stream]
Saving to: ‘vgg16_bn-6c64b313.pth’

vgg16_bn-6c64b313.p 100%[===================>] 527.87M  2.91MB/s    in 3m 20s  

2021-10-30 14:27:40 (2.64 MB/s) - ‘vgg16_bn-6c64b313.pth’ saved [553507836/553507836]



In [41]:
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load('vgg16_bn-6c64b313.pth'))

<All keys matched successfully>

In [42]:
print(vgg16.classifier[6].out_features) 

1000


In [43]:
for param in vgg16.features.parameters():
    param.require_grad = False

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] 
features.extend([nn.Linear(num_features, 149)]) 
vgg16.classifier = nn.Sequential(*features) 
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [44]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
initial_optimizer_state_dict = optimizer.state_dict()


In [45]:
optimizer =  optim.Adam(vgg16.parameters(), lr=0.01, weight_decay=0.0001)
initial_optimizer_state_dict = optimizer.state_dict()
checkpoint_interval = 2

In [46]:
def load_checkpoint(vgg16, optimizer, filename='checkpoint.pth'):
    start_epoch = 0
    print("=> loading checkpoint '{}'".format(filename))
    checkpoint = torch.load(filename)
    start_epoch = checkpoint['epoch']
    vgg16.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimiser'])
    print("=> loaded checkpoint '{}' (epoch {})"
              .format(filename, checkpoint['epoch']))
    return vgg16, optimizer, start_epoch

In [47]:
def train(vgg16, optimizer,start_epoch,epochs):
  start_epoch = 0
  vgg16.train()
  if use_cuda:
    vgg16.cuda()
  for epoch in range(start_epoch, epochs):
    for batch, (data, target) in enumerate(train_dataloader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()

      output = vgg16(data)
      _, preds = torch.max(output.data, 1)
      loss = criterion(output, target)
      # print(output)
      # print(target)
      # print(preds)
      loss.backward()
      optimizer.step()

      if batch % 10 == 0:
        done = batch * len(data)
        percentage = 100. * batch/ len(train_dataloader)
        print(f'Train Epoch: {epoch} [{done:5}/{len(train_dataloader.dataset)} ({percentage:3.0f}%)]  Loss: {loss.item():.6f}')
    if epoch % checkpoint_interval == 0:
      checkpoint = {'epoch' : epoch, 'model' : model.state_dict(), 'optimiser' : optimiser.state_dict()}
      checkpoint_path = f"checkpoints/vgg16_ckpt_{epoch}.pth"
      print(f"---- Saving checkpoint to: '{checkpoint_path}' ----")
      torch.save(checkpoint, checkpoint_path)
      return checkpoint_path
      break


In [48]:
checkpoint_path = None
start_epoch = 0
epochs = 10

checkpoint_path = train(vgg16,optimizer, start_epoch, epochs)
vgg16, optimizer, start_epoch = load_checkpoint(vgg16, optimizer,filename=checkpoint_path)

KeyboardInterrupt: 

In [ ]:
checkpoint_path = train(vgg16, optimizer,start_epoch,epochs)
vgg16, optimizer, start_epoch = load_checkpoint(vgg16, optimizer,filename=checkpoint_path)

In [ ]:
checkpoint_path = train(vgg16, optimizer,start_epoch,epochs)
vgg16, optimizer, start_epoch = load_checkpoint(vgg16, optimizer,filename=checkpoint_path)

In [ ]:
checkpoint_path = train(vgg16, optimizer,start_epoch,epochs)
vgg16, optimizer, start_epoch = load_checkpoint(vgg16, optimizer,filename=checkpoint_path)

In [ ]:
checkpoint_path = train(vgg16, optimizer,start_epoch,epochs)
vgg16, optimizer, start_epoch = load_checkpoint(vgg16, optimizer,filename=checkpoint_path)

In [ ]:
torch.save(vgg16.state_dict(), 'drive/MyDrive/sustainable_dataset/new_vgg16_bn.pth')

In [ ]:
def test(vgg16):
    vgg16.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_dataloader:
            data, target = data.to(device), target.to(device)
            output = vgg16(data)
            test_loss = criterion(output, target)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum().item()

        test_loss /= len(test_dataloader.dataset)
        accuracy = 100. * correct / len(test_dataloader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_dataloader.dataset)} ({accuracy:.2f}%)')
    return accuracy

In [ ]:
test(vgg16)